# Лабораторная работа 2, первая часть.

В первой части работы мы делаем простенький переводчик с украинского языка на русский на основе материалов из репозитория Яндекс ШАД.

Для начала подключим необходимые библиотеки:

In [2]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

Теперь загрузим векторные представления слов для украинского и русского языков.

Поскольку файлы имеют слишком большой размер, мы загружаем их частично.

In [4]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec", limit = 50000)
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec", limit = 50000)

Теперь мы можем задать слово из русского или украинского языка и посмотреть несколько его ближайших "соседей" в этом же языке.

In [5]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383153319358826),
 ('сентябрь', 0.9240027666091919),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930037021636963),
 ('апрель', 0.8729088306427002),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796871185303),
 ('февраль', 0.8401416540145874)]

In [6]:
uk_emb.most_similar([uk_emb["серпень"]], topn=10)

[('серпень', 0.9999998807907104),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

Теперь загрузим словари для тренировочных пар слов в качестве тренировочной и тестовой выборок:

In [8]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [9]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

### Отображение векторных представлений слов

Пусть $x_i \in R^d - $ представление слова $i$ в исходном языке, и $y_i \in R^d - $ представление его перевода. Наша задача $-$ отыскать линейное преобразование $W$, минимизирующее евклидово расстояние между векторами $W x_i$ и $y_i$ для некоторого подмножества векторных представлений слов. Известно, что искомое преобразование является линейной регрессией.

In [10]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression().fit(X_train, Y_train)

print(mapping.score(X_train, Y_train))
print(mapping.coef_)

0.6089079680467416
[[-0.04612916  0.0337263  -0.00562786 ...  0.05285112  0.10852949
   0.04657898]
 [ 0.01326142  0.06036477  0.04037627 ...  0.02810933 -0.00108582
  -0.00483881]
 [ 0.11702695 -0.02940938  0.03521974 ... -0.00296276  0.02643139
  -0.0257165 ]
 ...
 [ 0.04115694  0.00614955 -0.02377279 ... -0.0557879   0.10193649
  -0.00353186]
 [-0.05194057  0.03244646 -0.02714506 ...  0.05210593  0.08262951
  -0.06167889]
 [-0.05366036 -0.03043293 -0.00753437 ...  0.0489674   0.02959289
  -0.05236827]]


Посмотрим, каких соседей в русском языке имеет образ слова "серпень" после этого преобразования.

In [11]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8600566983222961),
 ('март', 0.8525829911231995),
 ('февраль', 0.846933126449585),
 ('июнь', 0.846840500831604),
 ('сентябрь', 0.8431644439697266),
 ('ноябрь', 0.8409683704376221),
 ('октябрь', 0.8386332392692566),
 ('июль', 0.8257951736450195),
 ('август', 0.8182761073112488),
 ('декабрь', 0.8174360990524292)]

Соседями по-прежнему оказались разные месяцы. С другой стороны, конкретный месяц правильно определить не получилось.

In [12]:
ru_emb.most_similar(mapping.predict(uk_emb["кіт"].reshape(1, -1)))

[('кот', 0.7644362449645996),
 ('щенок', 0.6685543060302734),
 ('пес', 0.6598378419876099),
 ('котенок', 0.6539851427078247),
 ('кролик', 0.6428782939910889),
 ('заяц', 0.6343851089477539),
 ('Кот', 0.6096946597099304),
 ('кошка', 0.6004424095153809),
 ('волк', 0.5991594791412354),
 ('тигр', 0.5911200046539307)]

А вот здесь все нормально, кiт оказался котом, а не каким-то другим зверем.

Теперь посчитаем долю украинских векторов, у которых после преобразования правильный перевод находится в числе первых N предложенных вариантов.

In [13]:
def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs): 
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x,_) in ru_emb.most_similar([mapped_vector],topn=topn)]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [14]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [15]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [16]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [17]:
print(precision_top1)
print(precision_top5)

0.6677966101694915
0.8338983050847457


Ясно, что чем большую часть векторных представлений загрузить, тем больше будут эти числа.

### Улучшение полученного результата 

Теперь потребуем, чтобы наше линейное преобразование было ортогональным.

In [18]:
def learn_transform(X, Y):
    U, s, V = np.linalg.svd(np.matmul(X_train.T,Y_train))
    W = np.matmul(U,V)
    return W

In [19]:
W = learn_transform(X_train, Y_train)

print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6779661016949152
0.8440677966101695


В результате точность немного увеличилась.

## Переводчик с украинского языка на русский

Теперь мы можем реализовать примитивный переводчик, который для каждого слова исходного языка будет отыскивать ближайшее слово в векторном пространстве целевого языка.

In [20]:
def translate(sentence):
    words = sentence.split(" ")
    translation = []
    for word in words:
        try:
            emb = uk_emb[word]
            translation.append(ru_emb.most_similar([np.matmul(emb, W)], topn=1)[0][0])           
        except:
            translation.append(word)
    return " ".join(translation)

In [21]:
print(translate("яка гарна дівка"))

она красивая дівка


In [22]:
print(translate("кіт зловив мишу"))

кот зловив мишу


In [23]:
print(translate("добре сало з горілкою , особливо з перцем"))

хорошо сало со горілкою , особенно со перцем


Получили посредственного качества перевод, опять же, из-за ограниченного объема векторных представлений.

In [24]:
with open("fairy_tale.txt", "r", encoding='UTF8') as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [25]:
uk_sentences

['лисичка - сестричка і вовк - панібрат',
 'як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !',
 'дай мені огня " .',
 'а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .',
 'прибігла на поле , аж там пасуть хлопці бичків .',
 'вона і каже їм : " ей , хлопці !',
 'проміняйте мені бичка - третячка за маковий пирожок " .',
 'тії согласились ; так вона їм говорить : " смотріть же , ви не їжте зараз сього пирожка , а тоді уже розломите , як я заведу бичка за могилку ; а то ви його ні за що не розломите " .',
 'бачите вже - лисичка таки собі була розумна , що хоть кого да обманить .',
 'тії хлопці так і зробили , а лисичка як зайшла за могилу , да зараз у ліс і повернула , 

In [26]:
def translate(sentence, src_emb, dst_emb):
    words = sentence.split(' ')
    translation = []
    for word in words:
        try:
            ru_word = dst_emb.most_similar([np.matmul(src_emb[word], W)], topn=1)[0][0]
            translation.append(ru_word)           
        except:
            translation.append(word)
    return ' '.join(translation)

def uk_ru_translate(sentence):
    return translate(sentence, uk_emb, ru_emb)

In [27]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, uk_ru_translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – панібрат

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , че и пошла раз к однії бабы добувать огню ; вошла во дом че и говорит : " хороший день тебе , бабусю !

src: дай мені огня " .
dst: дай мне огня " .

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а баба только что вийняла со печи пирожок со маком , сладкий , че и положила , чтобы он прохолов ; а лисичка ж и підгледала , че тілько что баба нахилилась во печь , чтобы достать огня , то лисичка сейчас ухватила пирожок че и драла со дома , че , біжучи , весь мак со его виїла , а туда мусор н

src: вона каже : " наловила , вовчику - братику ! "
dst: она говорит : " наловила , вовчику – братику ! "

src: а собі на думці : " подожди , і я зроблю з тобою таку штуку , як і ти зо мною " .
dst: а себе по мнении : " подожди , и мной сделаю со тобой такую штуку , как и ты За мной " .

src: - " як же ти ловила ? "
dst: – " как то ты ловила ? "

src: - " так , вовчику , уложила хвостик в ополонку , вожу тихенько да й кажу ; ловися , рибка , мала і велика !
dst: – " так , вовчику , уложила хвостик во ополонку , вожу тихенько че и говорю ; ловися , рибка , имела и большая !

src: коли хочеш , то і ти піди , налови собі " .
dst: когда хочу , то и ты піди , налови себе " .

src: він побіг да зробив так , як казала лисичка .
dst: он побіг че сделал так , как говорила лисичка .

src: а лисичка стала за деревом да й дивиться ; коли у вовчика зовсім хвостик примерз , вона тоді побігла в село да й кричить : " ідіть , люди , вбивайте вовка ! "
dst: а лисичка стала за деревом че и смотрит ; когд

Видим, что переводчик работает, пусть и не идеально.